# Fitbit Exploration
For an explanation on the variables, take a look at the [data dictionary created by Fitabase](https://www.fitabase.com/media/1546/fitabasedatadictionary.pdf).

## Sleep Sensitivity - 1 Variable
In this notebook we take a look at the individual variables that might be affecting sleep

In [1]:
import os
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from joypy import joyplot

# Data Import
Sleep data are divided into two primary datasets:

1. Sleep Summaries by Day (daily sleep)
2. Sleep Data by Minute (sleep stages)

In [8]:
daily_sleep = pd.read_csv("../data/processed/fitbit-sleep_daily-ux_s20.csv",parse_dates=["date","start_time","end_time"],infer_datetime_format=True)
# converting duration to something that makes more sense...
daily_sleep['tst'] = daily_sleep['duration_ms'] / 3600000
daily_sleep = daily_sleep[daily_sleep["main_sleep"] == True]
daily_sleep.drop(["minutes_to_sleep","main_sleep"],axis=1,inplace=True)
daily_sleep = daily_sleep[['beiwe', 'start_time', 'end_time', 'date','tst','duration_ms','minutes_after_wakeup', 'minutes_asleep', 'minutes_awake', 'time_in_bed', 'efficiency']]
daily_sleep.head()

,beiwe,start_time,end_time,date,tst,duration_ms,minutes_after_wakeup,minutes_asleep,minutes_awake,time_in_bed,efficiency
0,hfttkth7,2020-05-14 00:27:00,2020-05-14 07:13:00,2020-05-14,6.766667,24360000,0,379,27,406,97
1,hfttkth7,2020-05-14 23:53:30,2020-05-15 08:06:30,2020-05-15,8.216667,29580000,8,392,101,493,87
2,hfttkth7,2020-05-15 23:28:00,2020-05-16 04:57:00,2020-05-16,5.483333,19740000,7,287,42,329,95
3,hfttkth7,2020-05-17 02:01:30,2020-05-17 09:28:30,2020-05-17,7.450000,26820000,8,403,44,447,96
4,hfttkth7,2020-05-18 00:24:00,2020-05-18 07:20:00,2020-05-18,6.933333,24960000,0,351,65,416,92


In [9]:
sleep_stages = pd.read_csv("../data/processed/fitbit-sleep_stages-ux_s20.csv",parse_dates=["start_date","end_date","time"],infer_datetime_format=True)
sleep_stages.head()

,start_date,end_date,time,stage,time_at_stage,beiwe,value
0,2020-05-14,2020-05-14,2020-05-14 00:27:00,wake,510,hfttkth7,0
1,2020-05-14,2020-05-14,2020-05-14 00:35:30,light,420,hfttkth7,1
2,2020-05-14,2020-05-14,2020-05-14 00:42:30,deep,1590,hfttkth7,2
3,2020-05-14,2020-05-14,2020-05-14 01:09:00,light,1290,hfttkth7,1
4,2020-05-14,2020-05-14,2020-05-14 01:30:30,rem,840,hfttkth7,3


The [data dictionary](https://www.fitabase.com/media/1546/fitabasedatadictionary.pdf) for these variables can be quite enlightening as many of these variables are useless.

# Getting Features
Here we combine datasets across the Fitbit, EMAs, and Beacon

In [21]:
beacon = pd.read_csv("../data/processed/beacon-fb_ema_and_gps_filtered-ux_s20.csv",index_col=0,parse_dates=True,infer_datetime_format=True)
beacon.columns

Index(['lat', 'long', 'altitude', 'accuracy', 'tvoc', 'lux', 'no2', 'co',
       'co2', 'pm1_number', 'pm2p5_number', 'pm10_number', 'pm1_mass',
       'pm2p5_mass', 'pm10_mass', 'temperature_c', 'rh', 'beacon', 'beiwe',
       'fitbit', 'redcap', 'start_time', 'end_time'],
      dtype='object')

In [28]:
beacon_mean = pd.DataFrame()
for pt in beacon["beiwe"].unique():
    beacon_by_pt = beacon[beacon["beiwe"] == pt]
    ids = beacon_by_pt[["end_time","beacon","beiwe","fitbit","redcap"]]
    beacon_mean_by_pt = beacon_by_pt.groupby("start_time").mean()
    beacon_mean_by_pt["end_time"] = ids["end_time"].unique()
    for col in ids.columns[1:]:
        beacon_mean_by_pt[col] = ids[col][0]
    beacon_mean = beacon_mean.append(beacon_mean_by_pt)
    
beacon_mean

NameError: name 'mean' is not defined

In [11]:
beacon = pd.read_csv("../data/processed/fitbit_beiwe_beacon-sleep_summary-ux_s20.csv")
beacon.columns

Index(['date', 'duration_ms', 'efficiency', 'end_time', 'main_sleep',
       'minutes_after_wakeup', 'minutes_asleep', 'minutes_awake',
       'minutes_to_sleep', 'start_time', 'time_in_bed', 'content', 'stress',
       'lonely', 'sad', 'energy', 'tst', 'sol', 'naw', 'restful', 'beiwe',
       'beacon', 'fitbit', 'redcap'],
      dtype='object')

In [12]:
beacon = pd.read_csv("../data/processed/beacon-fb_ema_and_gps_filtered-ux_s20.csv")

In [15]:
beacon.columns

Index(['timestamp', 'lat', 'long', 'altitude', 'accuracy', 'tvoc', 'lux',
       'no2', 'co', 'co2', 'pm1_number', 'pm2p5_number', 'pm10_number',
       'pm1_mass', 'pm2p5_mass', 'pm10_mass', 'temperature_c', 'rh', 'beacon',
       'beiwe', 'fitbit', 'redcap', 'start_time', 'end_time'],
      dtype='object')